## Importing libraries

In [28]:
from typing import Dict, List, Optional, Union

import cirq
import numpy as np

## Creating classical data

In [2]:
is_low_rank_assumption_passed = False
N_TRIES_LIMIT = 10

i = 0
while not is_low_rank_assumption_passed and i <= N_TRIES_LIMIT:
    i += 1
    # Number of users
    m = 16
    # Number of products
    n = 8
    # Preference matrix
    P = np.random.randint(2, size=(m, n))
    
    # Rank of approximation
    k = 5
    
    U, S, V = np.linalg.svd(P)
    S[k:] = 0
    rank = np.sum(S > 0)
    is_low_rank_assumption_passed = rank == k
    if is_low_rank_assumption_passed:
        reconstruct_S = np.zeros((U.shape[0], V.shape[0]))
        np.fill_diagonal(reconstruct_S, S)
        approximation_matrix = U @ reconstruct_S @ V
        epsilon = np.linalg.norm(P - approximation_matrix) / np.linalg.norm(P)

if i == N_TRIES_LIMIT + 1:
    raise RuntimeError("Couldn't find an approximation matrix with the given parameters.")

print(f"P = {P}")

P = [[1 1 0 1 1 1 0 1]
 [0 1 0 0 1 1 1 1]
 [0 0 0 0 0 1 0 1]
 [0 1 0 0 1 0 1 0]
 [1 0 1 0 0 1 1 1]
 [1 0 1 0 1 1 0 1]
 [1 1 0 1 0 1 1 0]
 [1 1 0 0 0 1 0 1]
 [0 0 0 1 0 1 1 0]
 [1 0 1 1 1 0 1 1]
 [1 1 0 0 0 1 1 0]
 [1 1 1 1 1 1 0 1]
 [0 1 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0]
 [1 1 0 1 0 1 1 1]
 [0 1 1 0 0 1 1 1]]


## Creating data structure

In [29]:
class _Leaf:
    def __init__(self, value: float, positive_sign: bool):
        self.value: float = value
        self.positive_sign: bool = positive_sign
    
    def is_leaf(self):
        return True

class _BinaryTree:
    def __init__(self, depth: int, value: float = 0.):
        self.value: float = value
        self.depth: int = depth
        self.left: Union[_BinaryTree, _Leaf] = _Leaf(-1, True)
        self.right: Union[_BinaryTree, _Leaf] = _Leaf(-1, True)
    
    def __getitem__(self, key: int):      
        if key - 1 < 0:
            raise KeyError(f"Invalid key: {key} (must be strictly positive).")
            
        binary_path: str = bin(key - 1)[2:]
        binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
        if len(binary_path) != self.depth:
            raise KeyError(f"Invalid key for storage: {position} (too long).")
        
        def get_aux(binary_tree: _BinaryTree, binary_path: str):
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            
            if len(binary_path) == 1:
                if position == "0":
                    if binary_tree.left.value == -1:
                        raise KeyError()
                    return binary_tree.left.value, binary_tree.left.positive_sign
                if binary_tree.right.value == -1:
                    raise KeyError()
                return binary_tree.right.value, binary_tree.right.positive_sign

            if position == "0":
                if isinstance(binary_tree.left, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.left, binary_path[1:])
            else:
                if isinstance(binary_tree.right, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.right, binary_path[1:])
        
        try:
            return get_aux(self, binary_path)
        except KeyError:
            raise KeyError(f"No value stored for key: {key}.")
    
    def __str__(self):
        return f"_BinaryTree(depth={self.depth}, value={self.value:.2f})"
        
    def is_leaf(self) -> bool:
        return False
    
    def store(self, position: int, value: float, positive_sign: bool) -> None:
        def store_aux(binary_tree: _BinaryTree, binary_path: str, value: float, positive_sign: bool) -> None:
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            
            if len(binary_path) == 1:
                if position == "0":
                    binary_tree.left = _Leaf(value, positive_sign)
                else:
                    binary_tree.right = _Leaf(value, positive_sign)
            else:
                if position == "0":
                    if isinstance(binary_tree.left, _Leaf):
                        binary_tree.left = _BinaryTree(len(binary_path) - 1)

                    binary_tree.left.value += value
                    store_aux(binary_tree.left, binary_path[1:], value, positive_sign)
                else:
                    if isinstance(binary_tree.right, _Leaf):
                        binary_tree.right = _BinaryTree(len(binary_path) - 1)

                    binary_tree.right.value += value
                    store_aux(binary_tree.right, binary_path[1:], value, positive_sign)
        
        if isinstance(position, int):
            if position - 1 < 0:
                raise ValueError(f"Invalid position: {position} (must be strictly positive).")
                
            binary_path = bin(position - 1)[2:]
            binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
            if len(binary_path) != self.depth:
                raise KeyError(f"Invalid key for storage: {position} (too long).")
            
            store_aux(self, binary_path, value, positive_sign)
            self.value = self.left.value + self.right.value
        else:
            raise TypeError(f"Incorrect type for position: {type(position)} (int expected).")
            

class DataStructure:
    def __init__(self, array: np.ndarray):
        if not isinstance(array, np.ndarray):
            raise TypeError(f"Can only store np.ndarray ({type(array)} given).")
        if len(array.shape) > 2:
            raise ValueError(f"Can't store arrays with more than 2 dimensions (shape {array.shape} given).")
        if len(array.shape) == 1:
            array = array.reshape((1, array.shape[0]))

        self.m: int = array.shape[0]
        self.n: int = array.shape[1]
        self.trees_depth = int(.5 + np.log2(n))
        self._trees: Dict[int, _BinaryTree] = {}
        self._store_init(array)
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return self._trees[key]
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(tuple)} given).")
            if self._trees.get(key[0]) is None:
                self._trees[i] = _BinaryTree(self.trees_depth)
                
            return self._trees[key[0]][key[1]]
        raise KeyError(f"Not a representable address: {key}.")
    
    def __repr__(self):
        return f"DataStructure({[str(self._trees[x]) for x in self._trees]})"
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(key)} given).")
            if not (isinstance(value, tuple) or isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64)):
                raise TypeError(f"Not an acceptable type to store: {type(value)} (numeric type or tuple expected).")
            if isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64):
                self._store(key[0], key[1], value ** 2, value >= 0)
                return
            if len(value) != 2:
                raise ValueError(f"value must be a tuple of length 2 (length {len(value)} given).")
            if not isinstance(value[0], float):
                raise ValueError(f"The first element of value must be a float ({type(value[0])} given).")
            if value[0] < 0 or value[0] > 1:
                raise ValueError(f"The first element of value must be between 0 and 1 ({value[0]} given).")
            if not isinstance(value[1], bool):
                raise ValueError(f"The second element of value must be a boolean ({type(value[1])} given).")
                
            self._store(key[0], key[1], value[0], value[1])
        else:
            raise TypeError(f"Invalid address given: {key}.")
    
    def _store(self, i: int, j: int, value: float, positive_sign: bool) -> None:
        if self._trees.get(i) is None:
            self._trees[i] = _BinaryTree(self.trees_depth)
            
        self._trees[i].store(j, value, positive_sign)
    
    def _store_init(self, array: np.ndarray) -> None:        
        for i in range(array.shape[0]):
            for j in range(array.shape[1]):
                self[i + 1, j + 1] = array[i, j]
    
    def reconstruct(self, circuit: cirq.Cicuit, qubits: List[qubits], reset:bool = False) -> None:
        """
            :param circuit: The Circuit to add the operations to
            :param qubits: A List of qubits to create the state on.
            :param reset: A boolean indicating whether the qubits need to be set in the 0 state
        """
        
        if self.m != 1:
            raise ValueError(f"Can't reconstruct a matrix.")
        if reset:
            raise NotImplementedError(f"Can't reset the qubits for now.")
        if len(qubits) != self.trees_depth:
            raise ValueError(f"Incorrect number of qubits (expected {self.trees_depth}, got {len(qubits)}).")
        
        squared_norm: float = self[1].value
        theta: float = (np.arccos(np.sqrt(self[1].left.value / squared_norm)) + np.arcsin(np.sqrt(self[1].right.value / squared_norm))) / 2
        circuit.append(cirq.ry(theta)(qubits[0]))

In [30]:
ds_P = DataStructure(P)